In [20]:
import arcpy

# Setting the workspace
arcpy.env.workspace = r'\\Mac\Home\gigi_git\OptimizedRoutes\VRP.gdb'

# Enabling overwrite output
arcpy.env.overwriteOutput = True


In [71]:
import os
os.getcwd()

'\\\\Mac\\Home\\gigi_git\\OptimizedRoutes'

In [28]:
import arcgis
from arcgis.gis import GIS
import pandas as pd
import datetime
from arcgis import geocoding
from arcgis.features import Feature, FeatureSet

In [34]:
print("ArcGIS Online Org account")    
gis = GIS('home')
print("Logged in as " + str(gis.properties.user.username))

ArcGIS Online Org account
Logged in as jis310_Harvard_CGA


In [35]:
orders_csv = "PythonNotebook\\Addresses.csv"
orders_df = pd.read_csv(orders_csv)
orders_sdf = pd.DataFrame.spatial.from_df(orders_df, "Address")
orders_fset = orders_sdf.spatial.to_featureset()
orders_fset

<FeatureSet> 483 features

In [37]:
out_path = r'\\Mac\Home\gigi_git\OptimizedRoutes\VRP.gdb'
out_name = "orders_feature_class"
orders_fset.save(out_path, out_name)

'\\\\Mac\\Home\\gigi_git\\OptimizedRoutes\\VRP.gdb\\orders_feature_class'

In [38]:
routes_csv = "PythonNotebook\\routes.csv"

# Read the csv file
routes_df = pd.read_csv(routes_csv, parse_dates=["EarliestStartTime", "LatestStartTime"], date_parser=pd.to_datetime)
routes_df["EarliestStartTime"] = routes_df["EarliestStartTime"].astype("int64") / 10 ** 6
routes_df["LatestStartTime"] = routes_df["LatestStartTime"].astype("int64") / 10 ** 6
routes_fset = arcgis.features.FeatureSet.from_dataframe(routes_df)
routes_df.head(5)

,ObjectID *,Shape *,Name,Description,StartDepotName,EndDepotName,StartDepotServiceTime,EndDepotServiceTime,EarliestStartTime,LatestStartTime,ArriveDepartDelay,Capacity_1,Capacity_2,Capacity_3,Capacity_4,Capacity_5,Capacity_6,Capacity_7,Capacity_8,Capacity_9,FixedCost,CostPerUnitTime,CostPerUnitDistance,OvertimeStartTime,CostPerUnitOvertime,MaxOrderCount,MaxTotalTime,MaxTotalTravelTime,MaxTotalDistance,AssignmentRule,ViolatedConstraint_1,ViolatedConstraint_2,ViolatedConstraint_3,ViolatedConstraint_4,OrderCount,TotalCost,RegularTimeCost,OvertimeCost,DistanceCost,TotalTime,TotalOrderServiceTime,TotalBreakServiceTime,TotalTravelTime,TotalDistance,StartTime,EndTime,StartTimeUTC,EndTimeUTC,TotalWaitTime,TotalViolationTime,RenewalCount,TotalRenewalServiceTime,Shape_Length
0,1,Polyline M,Route1,<Null>,"5110 Arundel Rakaia Gorge Road, Alford Forest,...","5110 Arundel Rakaia Gorge Road, Alford Forest,...",<Null>,<Null>,1.698872e+12,1.698872e+12,<Null>,60,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,0.5,<Null>,452,10,60,480,<Null>,<Null>,Include,<Null>,<Null>,<Null>,<Null>,54,273.637871,226.000000,47.637871,0,456.763787,226,0,230.763787,165.571912,11/1/2023 21:00,11/2/2023 4:36,11/1/2023 8:00,11/1/2023 15:36,0,0,0,0,3.013565
1,2,Polyline M,Route2,<Null>,"5110 Arundel Rakaia Gorge Road, Alford Forest,...","5110 Arundel Rakaia Gorge Road, Alford Forest,...",<Null>,<Null>,1.698872e+12,1.698872e+12,<Null>,60,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,0.5,<Null>,452,10,60,480,<Null>,<Null>,Include,<Null>,<Null>,<Null>,<Null>,54,215.324192,215.324192,0.000000,0,430.648384,228,0,202.648384,146.270688,11/1/2023 21:00,11/2/2023 4:10,11/1/2023 8:00,11/1/2023 15:10,0,0,0,0,2.653510
2,3,Polyline M,Route3,<Null>,"5110 Arundel Rakaia Gorge Road, Alford Forest,...","5110 Arundel Rakaia Gorge Road, Alford Forest,...",<Null>,<Null>,1.698872e+12,1.698872e+12,<Null>,60,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,0.5,<Null>,452,10,60,480,<Null>,<Null>,Include,<Null>,<Null>,<Null>,<Null>,57,230.339501,226.000000,4.339501,0,452.433950,234,0,218.433950,154.595579,11/1/2023 21:00,11/2/2023 4:32,11/1/2023 8:00,11/1/2023 15:32,0,0,0,0,2.787192
3,4,Polyline M,Route4,<Null>,"5110 Arundel Rakaia Gorge Road, Alford Forest,...","5110 Arundel Rakaia Gorge Road, Alford Forest,...",<Null>,<Null>,1.698872e+12,1.698872e+12,<Null>,60,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,0.5,<Null>,452,10,60,480,<Null>,<Null>,Include,<Null>,<Null>,<Null>,<Null>,51,222.225255,222.225255,0.000000,0,444.450510,222,0,222.450510,152.825985,11/1/2023 21:00,11/2/2023 4:24,11/1/2023 8:00,11/1/2023 15:24,0,0,0,0,2.814031
4,5,Polyline M,Route5,<Null>,"5110 Arundel Rakaia Gorge Road, Alford Forest,...","5110 Arundel Rakaia Gorge Road, Alford Forest,...",<Null>,<Null>,1.698872e+12,1.698872e+12,<Null>,60,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,0.5,<Null>,452,10,60,480,<Null>,<Null>,Include,<Null>,<Null>,<Null>,<Null>,53,233.217770,226.000000,7.217770,0,452.721777,226,0,226.721777,157.416889,11/1/2023 21:00,11/2/2023 4:32,11/1/2023 8:00,11/1/2023 15:32,0,0,0,0,2.806011


In [50]:
routes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ObjectID *               9 non-null      int64  
 1   Shape *                  9 non-null      object 
 2   Name                     9 non-null      object 
 3   Description              9 non-null      object 
 4   StartDepotName           9 non-null      object 
 5   EndDepotName             9 non-null      object 
 6   StartDepotServiceTime    9 non-null      object 
 7   EndDepotServiceTime      9 non-null      object 
 8   EarliestStartTime        9 non-null      float64
 9   LatestStartTime          9 non-null      float64
 10  ArriveDepartDelay        9 non-null      object 
 11  Capacity_1               9 non-null      int64  
 12  Capacity_2               9 non-null      object 
 13  Capacity_3               9 non-null      object 
 14  Capacity_4               9 non

In [51]:
out_name = "routes_feature_class"
routes_fset.save(out_path, out_name)

'\\\\Mac\\Home\\gigi_git\\OptimizedRoutes\\VRP.gdb\\routes_feature_class'

In [52]:
print(routes_fset)

{"features": [{"attributes": {"ObjectID *": 1, "Shape *": "Polyline M", "Name": "Route1", "Description": "<Null>", "StartDepotName": "5110 Arundel Rakaia Gorge Road, Alford Forest, Mount Somers, Canterbury, 7771", "EndDepotName": "5110 Arundel Rakaia Gorge Road, Alford Forest, Mount Somers, Canterbury, 7771", "StartDepotServiceTime": "<Null>", "EndDepotServiceTime": "<Null>", "EarliestStartTime": 1698872400000.0, "LatestStartTime": 1698872400000.0, "ArriveDepartDelay": "<Null>", "Capacity_1": 60, "Capacity_2": "<Null>", "Capacity_3": "<Null>", "Capacity_4": "<Null>", "Capacity_5": "<Null>", "Capacity_6": "<Null>", "Capacity_7": "<Null>", "Capacity_8": "<Null>", "Capacity_9": "<Null>", "FixedCost": "<Null>", "CostPerUnitTime": 0.5, "CostPerUnitDistance": "<Null>", "OvertimeStartTime": 452, "CostPerUnitOvertime": 10, "MaxOrderCount": 60, "MaxTotalTime": 480, "MaxTotalTravelTime": "<Null>", "MaxTotalDistance": "<Null>", "AssignmentRule": "Include", "ViolatedConstraint_1": "<Null>", "Viola

### 5. Create Depots Layer with an Address

In [42]:
depot_geocoded_fs = geocoding.geocode("5110 Arundel Rakaia Gorge Road, Alford Forest, Mount Somers, Canterbury, 7771", 
                                      as_featureset=True, max_locations=1)
depot_geocoded_fs.features[0].attributes["Name"] = "5110 Arundel Rakaia Gorge Road, Alford Forest, Mount Somers, Canterbury, 7771"
depot_geocoded_fs

<FeatureSet> 1 features

AttributeError: 'FeatureSet' object has no attribute 'head'

In [40]:
depot_geocoded_fs.features
depots_fset = depot_geocoded_fs.sdf.spatial.to_featureset()
depot_geocoded_fs

<FeatureSet> 1 features

In [47]:
out_name = "depots_feature_class"
depot_geocoded_fs.save(out_path, out_name)

'\\\\Mac\\Home\\gigi_git\\OptimizedRoutes\\VRP.gdb\\depots_feature_class'

### 6. Draw the Depots and Orders in map

In [45]:
# Create a map instance to visualize inputs in map
map_view_inputs = gis.map('Christchurch, New Zealand', zoomlevel=8)
map_view_inputs

MapView(layout=Layout(height='400px', width='100%'))

In [46]:
# Visualize order and depot locations with symbology
map_view_inputs.draw(orders_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","color": [76,115,0,255],"size": 8})
map_view_inputs.draw(depot_geocoded_fs, symbol={"type": "esriSMS","style": "esriSMSSquare","color": [255,115,0,255], "size": 10})

### 7. Solve VRP

In [54]:
%%time
today = datetime.datetime.now()
from arcgis.network.analysis import solve_vehicle_routing_problem

results = solve_vehicle_routing_problem(orders= orders_fset,
                                        depots = depots_fset,
                                        routes = routes_fset, 
                                        save_route_data='true',
                                        populate_directions='true',
                                        travel_mode="Driving Time",
                                        default_date=today)

print('Analysis succeeded? {}'.format(results.solve_succeeded))

WARNING 030194: Data values longer than 500 characters for field [Routes:Description] are truncated.
WARNING 030194: Data values longer than 500 characters for field [Routes:StartDepotName] are truncated.
WARNING 030194: Data values longer than 500 characters for field [Routes:EndDepotName] are truncated.


Analysis succeeded? True
Wall time: 40.7 s


In [64]:
df = pd.DataFrame([f.attributes for f in results.out_routes.features])
print(df)

   ObjectID  ...                                              SHAPE
0         1  ...  {'paths': [[[171.5234709550001, -43.6085045059...
1         2  ...  {'paths': [[[171.5234709550001, -43.6085045059...
2         3  ...  {'paths': [[[171.5234709550001, -43.6085045059...
3         4  ...  {'paths': [[[171.5234709550001, -43.6085045059...
4         5  ...  {'paths': [[[171.5234709550001, -43.6085045059...
5         6  ...  {'paths': [[[171.5234709550001, -43.6085045059...
6         7  ...  {'paths': [[[171.5234709550001, -43.6085045059...
7         8  ...  {'paths': [[[171.5234709550001, -43.6085045059...
8         9  ...  {'paths': [[[171.5234709550001, -43.6085045059...

[9 rows x 27 columns]


### 8. Look into the Results

In [55]:
# Display the output routes in a pandas dataframe.
out_routes_df = results.out_routes.sdf
out_routes_df[['Name','OrderCount','StartTime','EndTime','TotalCost','TotalDistance','TotalTime','TotalTravelTime','StartTimeUTC','EndTimeUTC']]

,Name,OrderCount,StartTime,EndTime,TotalCost,TotalDistance,TotalTime,TotalTravelTime,StartTimeUTC,EndTimeUTC
0,Route1,47,2023-11-14 08:00:00,2023-11-14 11:26:11.727000,103.097723,154.987466,206.195446,206.195446,2023-11-13 19:00:00,2023-11-13 22:26:11.727000
1,Route2,53,2023-11-14 08:00:00,2023-11-14 10:45:41.372999,82.844771,126.225504,165.689542,165.689542,2023-11-13 19:00:00,2023-11-13 21:45:41.372999
2,Route3,60,2023-11-14 08:00:00,2023-11-14 11:17:14.848999,98.623745,144.058168,197.24749,197.24749,2023-11-13 19:00:00,2023-11-13 22:17:14.848999
3,Route4,37,2023-11-14 08:00:00,2023-11-14 11:00:56.698999,90.472491,140.229108,180.944981,180.944981,2023-11-13 19:00:00,2023-11-13 22:00:56.698999
4,Route5,60,2023-11-14 08:00:00,2023-11-14 11:20:51.556000,100.42963,141.569693,200.859261,200.859261,2023-11-13 19:00:00,2023-11-13 22:20:51.556000
5,Route6,60,2023-11-14 08:00:00,2023-11-14 11:32:55.811000,106.465091,150.074354,212.930182,212.930182,2023-11-13 19:00:00,2023-11-13 22:32:55.811000
6,Route7,60,2023-11-14 08:00:00,2023-11-14 11:43:35.759000,111.797993,154.35949,223.595985,223.595985,2023-11-13 19:00:00,2023-11-13 22:43:35.759000
7,Route8,53,2023-11-14 08:00:00,2023-11-14 11:37:36.964999,108.808044,152.885388,217.616088,217.616088,2023-11-13 19:00:00,2023-11-13 22:37:36.964999
8,Route9,53,2023-11-14 08:00:00,2023-11-14 11:09:42.910000,94.857587,137.55047,189.715174,189.715174,2023-11-13 19:00:00,2023-11-13 22:09:42.910000


In [56]:
out_stops_df = results.out_stops.sdf
out_stops_df[['Name','RouteName','Sequence','ArriveTime','DepartTime']].sort_values(by=['RouteName',
                                                                                       'Sequence'])

,Name,RouteName,Sequence,ArriveTime,DepartTime
483,"5110 Arundel Rakaia Gorge Road, Alford Forest,...",Route1,1,2023-11-14 08:00:00.000000,2023-11-14 08:00:00.000000
435,Location 436,Route1,2,2023-11-14 09:19:00.384999,2023-11-14 09:19:00.384999
437,Location 438,Route1,3,2023-11-14 09:19:06.365999,2023-11-14 09:19:06.365999
438,Location 439,Route1,4,2023-11-14 09:19:25.971000,2023-11-14 09:19:25.971000
441,Location 442,Route1,5,2023-11-14 09:20:15.412000,2023-11-14 09:20:15.412000
...,...,...,...,...,...
272,Location 273,Route9,51,2023-11-14 09:59:18.305999,2023-11-14 09:59:18.305999
273,Location 274,Route9,52,2023-11-14 10:00:05.967000,2023-11-14 10:00:05.967000
262,Location 263,Route9,53,2023-11-14 10:03:30.408999,2023-11-14 10:03:30.408999
173,Location 174,Route9,54,2023-11-14 10:08:51.588999,2023-11-14 10:08:51.588999


### 9. Display Output Routes

In [58]:
# Create a map instance to visualize inputs in map
map_view_outputs = gis.map('Christchurch, New Zealand', zoomlevel=8)
map_view_outputs

MapView(layout=Layout(height='400px', width='100%'))

In [59]:
#Visusalize the inputsn with different symbols
map_view_outputs.draw(orders_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSCircle",
                                           "color": [76,115,0,255],"size": 8})
map_view_outputs.draw(depots_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSSquare",
                                           "color": [255,115,0,255], "size": 10})

#Visualize the first route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[0])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [0,100,240,255],"size":10})

#Visualize the second route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[1])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [255,0,0,255],"size":10})

In [65]:
import random
from arcgis.features import FeatureSet

# Function to generate a random color
def generate_random_color():
    return [random.randint(0, 255) for _ in range(3)] + [255]  # RGB color with full opacity

# Visualize the orders and depots
map_view_outputs.draw(orders_fset, symbol={"type": "esriSMS", "style": "esriSMSCircle", "color": [76, 115, 0, 255], "size": 8})
map_view_outputs.draw(depots_fset, symbol={"type": "esriSMS", "style": "esriSMSSquare", "color": [255, 115, 0, 255], "size": 10})

# Iterate through all routes
for route in results.out_routes.features:
    # Create a FeatureSet for each route
    out_routes_fset = FeatureSet([route])

    # Generate a random color for the route
    color = generate_random_color()

    # Draw the route on the map
    map_view_outputs.draw(out_routes_fset, symbol={"type": "esriSLS", "style": "esriSLSSolid", "color": color, "size": 10})


In [70]:
# Assuming map_view_outputs is your map object where routes are drawn
webmap_properties = {'title': 'Optimizing VRP Routes',
                     'snippet': 'Web map showing VRP routes',
                     'tags': 'VRP, routes'}

# Update the thumbnail path
thumbnail_path = r'\\Mac\Home\gigi_git\OptimizedRoutes\Screenshot 2023-11-14 at 12.07.47 AM.png'  # Update this path

# Save the map as a web map
try:
    web_map_item = map_view_outputs.save(webmap_properties, thumbnail=thumbnail_path, folder=None)
    print(f"Web Map created: {web_map_item.homepage}")
except Exception as e:
    print(f"Error saving the web map: {e}")

Web Map created: https://www.arcgis.com//home/item.html?id=9336d7f06d7c4ed5b9f80a5178a78f41


### 10. Create Route Layers for Navigation

In [63]:
route_data = results.out_route_data.download('.')
route_data_item = gis.content.add({"type": "File Geodatabase"}, route_data)
route_layers = arcgis.features.analysis.create_route_layers(route_data_item, 
                                                            delete_route_data_item=True)
for route_layer in route_layers:
    route_layer.share(org=True)
    display(route_layer.homepage)
    display(route_layer)

{"cost": 0}


'https://www.arcgis.com//home/item.html?id=22acfa2a144449b3877a972673332a3e'

<Item title:"Route1" type:Feature Collection owner:jis310_Harvard_CGA>

'https://www.arcgis.com//home/item.html?id=ea583562d4a04274ad889ac396de6952'

<Item title:"Route2" type:Feature Collection owner:jis310_Harvard_CGA>

...

<Item title:"Route7" type:Feature Collection owner:jis310_Harvard_CGA>

'https://www.arcgis.com//home/item.html?id=4f2f73417a3f4c6e97297d918deb3be4'

<Item title:"Route8" type:Feature Collection owner:jis310_Harvard_CGA>

'https://www.arcgis.com//home/item.html?id=698fc7eb1eac4703abe5ab0379b8a732'

<Item title:"Route9" type:Feature Collection owner:jis310_Harvard_CGA>